In [1]:
import os
os.getcwd()

'c:\\Desktop\\AgenticSQL\\code'

In [2]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [3]:
## download chinook database

## it is a small database with tables related to music, albums, track etc. so we can use it with sqlite.

import requests

url = "https://github.com/tharani001/MyDatasets/raw/refs/heads/main/db_samples/Chinook.db"

response = requests.get(url)

if response.status_code == 200:
    with open("../database/Chinook.db", "wb") as file:
        file.write(response.content)
    print("Chinook.db downloaded successfully.")
else:
    print(f"Failed to download Chinook.db. Status code: {response.status_code}")

Chinook.db downloaded successfully.


### Load Database with Langchain SQLDatabase

In [6]:
# sqlite is small database that provides similar functionality to mysql and postgresql.

from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///../database/Chinook.db")
# print(db.table_names())
print(db.dialect)
print(db.get_table_info()) 

sqlite

CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCE

In [8]:
print(db.run("SELECT * FROM Album LIMIT 5;"))

[(1, 'For Those About To Rock We Salute You', 1), (2, 'Balls to the Wall', 2), (3, 'Restless and Wild', 2), (4, 'Let There Be Rock', 1), (5, 'Big Ones', 3)]


### LLM initialization

In [9]:
# Qwen2.5 is twice the size of LLAMA3.2 model but it performs better on code related tasks.

from langchain_ollama import ChatOllama

model = "qwen2.5:7b"

base_url = "http://localhost:11434"

llm  = ChatOllama(model=model, base_url=base_url)

llm

ChatOllama(model='qwen2.5:7b', base_url='http://localhost:11434')

### Building Graph State 

In [10]:
# Graph is workflow that controls the how an agent thinks and acts by managing the flow of states between the nodes.

# state is just a dictionary that contains the messages.

# each node takes a state and returns a new state. it gets updated with the new state.

from typing import TypedDict

class State(TypedDict):
    question : str
    query : str
    result : str
    answer : str